In [1]:
import pandas as pd 
import tensorflow as tf
from transformers import BertTokenizer ,TFBertForSequenceClassification,pipeline
from sklearn.metrics import accuracy_score,classification_report

In [2]:
df = pd.read_csv('enhanced_event_reviews.csv')
df.head()

,review,sentiment
0,"No proper seating arrangement, very uncomforta...",negative
1,The app for the event kept crashing. I wouldn'...,negative
2,The live music performance was mesmerizing. Th...,positive
3,The event was absolutely fantastic! The atmosp...,positive
4,Great networking opportunities and friendly st...,positive


In [3]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [4]:
sentiment_pipeline = pipeline("sentiment-analysis",model=model,tokenizer =tokenizer,framework ="tf")

Device set to use 0


In [15]:
predictions=[]
for review in df['review']:
    result = sentiment_pipeline(review)[0]
    label = result['label']

    # Map star ratings to your categories
    if label in ['1 star', '2 stars']:
        predictions.append('negative')
    elif label == '3 stars':
        predictions.append('neutral')
    else:  # 4 or 5 stars
        predictions.append('positive')

In [16]:
predictions

['negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'neutral',
 'negative',
 'positive',
 'positive',
 'negative',
 'neutral',
 'negative',
 'positive',
 'positive',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'neutral',
 'positive',
 'negative',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'negative',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'negati

In [17]:
df['predicted_sentiment'] = predictions

In [18]:
accuracy = accuracy_score(df['sentiment'], df['predicted_sentiment'])
report = classification_report(df['sentiment'], df['predicted_sentiment'])

print(f"✅ Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)

✅ Accuracy: 0.8949

Classification Report:
               precision    recall  f1-score   support

    negative       0.82      0.88      0.85       333
     neutral       0.87      0.81      0.84       333
    positive       1.00      1.00      1.00       333

    accuracy                           0.89       999
   macro avg       0.90      0.89      0.89       999
weighted avg       0.90      0.89      0.89       999



In [5]:
### fine tuning the model
def encode_reviews(reviews, tokenizer, max_len=128):
    return tokenizer(list(reviews), padding='max_length', truncation=True, max_length=max_len, return_tensors="tf")

In [6]:
X = encode_reviews(df['review'].tolist(), tokenizer)

In [7]:
sentiment_map = {
    'negative': 0,  # 1-2 stars
    'neutral': 2,   # 3 stars
    'positive': 4    # 4-5 stars
}


In [8]:
y = df['sentiment'].map(sentiment_map).values

In [9]:
y

array([0, 0, 4, 4, 4, 0, 4, 4, 4, 0, 0, 4, 2, 2, 4, 2, 0, 0, 0, 0, 0, 0,
       0, 4, 0, 2, 2, 4, 4, 2, 2, 0, 4, 4, 2, 0, 2, 0, 4, 2, 4, 4, 2, 2,
       2, 0, 4, 2, 0, 0, 4, 4, 0, 4, 0, 2, 4, 0, 4, 2, 2, 0, 2, 4, 0, 4,
       4, 2, 2, 0, 4, 0, 0, 0, 2, 4, 2, 0, 0, 0, 4, 2, 4, 4, 0, 4, 0, 2,
       0, 2, 0, 2, 2, 4, 2, 4, 2, 2, 0, 2, 2, 4, 2, 4, 0, 4, 2, 2, 4, 2,
       0, 4, 0, 0, 0, 2, 0, 4, 4, 2, 0, 4, 2, 2, 0, 4, 4, 0, 2, 0, 4, 0,
       2, 4, 2, 4, 2, 4, 4, 2, 2, 4, 4, 0, 2, 4, 2, 2, 0, 4, 4, 0, 4, 2,
       4, 0, 0, 0, 2, 0, 0, 4, 0, 4, 4, 0, 4, 2, 0, 4, 2, 2, 4, 4, 4, 4,
       0, 2, 0, 0, 0, 2, 4, 0, 2, 4, 2, 2, 0, 2, 4, 0, 4, 0, 2, 2, 0, 4,
       4, 4, 4, 4, 2, 0, 4, 4, 0, 4, 2, 2, 0, 4, 0, 4, 4, 2, 2, 4, 2, 2,
       0, 2, 0, 2, 2, 4, 4, 0, 2, 0, 0, 0, 4, 0, 2, 2, 0, 0, 4, 4, 0, 4,
       2, 4, 4, 0, 0, 4, 2, 0, 0, 2, 4, 0, 2, 4, 2, 0, 0, 2, 4, 0, 0, 0,
       0, 2, 2, 2, 4, 4, 4, 2, 4, 4, 0, 0, 2, 0, 2, 0, 0, 2, 4, 2, 2, 2,
       4, 4, 2, 4, 2, 4, 2, 0, 0, 4, 0, 0, 2, 2, 2,

In [11]:
BATCH_SIZE = 16
train_dataset = tf.data.Dataset.from_tensor_slices((dict(X), y)).shuffle(len(y)).batch(BATCH_SIZE)

In [12]:
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=5)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
EPOCHS = 2
model.fit(train_dataset, epochs=EPOCHS)

Epoch 1/2


63/63 [==============================] - 323s 5s/step - loss: 0.7217 - accuracy: 0.7347
Epoch 2/2
63/63 [==============================] - 293s 5s/step - loss: 0.0071 - accuracy: 1.0000


In [15]:
model

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# ✅ Inference on the test dataset
predictions = []
true_labels = []

for review, label in zip(df['review'], df['sentiment']):
    inputs = tokenizer(review, return_tensors="tf", padding=True, truncation=True)
    outputs = model(**inputs)
    pred_label = tf.argmax(outputs.logits, axis=1).numpy()[0]
    
    predictions.append(pred_label)
    true_labels.append(label)

# ✅ Map predictions back to star labels
label_map = {
    0: 'negative',   # 1-2 stars
    1: 'negative',   
    2: 'neutral',     # 3 stars
    3: 'positive',     # 4-5 stars
    4: 'positive'
}

predicted_sentiment = [label_map[p] for p in predictions]

# ✅ Calculate accuracy and classification report
accuracy = accuracy_score(df['sentiment'], predicted_sentiment)
report = classification_report(df['sentiment'], predicted_sentiment)

print(f"✅ Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)

In [19]:
# ✅ Map predictions back to star labels
label_map = {
    0: 'negative',   # 1-2 stars
    1: 'negative',   
    2: 'neutral',     # 3 stars
    3: 'positive',     # 4-5 stars
    4: 'positive'
}
new_reviews = [
    "Attending this event was an absolute delight! The organizers did a phenomenal job ensuring everything was seamless, from the registration process to the closing ceremony. The speakers were incredibly insightful, offering fresh perspectives that kept the audience engaged. The ambiance was vibrant, and the catering was top-notch. I particularly loved the networking opportunities, as I met professionals who provided valuable insights into my field. Overall, a well-organized, highly enjoyable experience that I would definitely recommend!",

    "The event had its ups and downs. While I appreciated the effort put into organizing it, I felt some aspects could have been improved. The keynote speakers were great, but the breakout sessions felt repetitive. The venue was decent, though the seating arrangements were a bit cramped. The networking session was useful, but I wish there were more structured ways to interact with other attendees. It wasn’t the worst event I’ve been to, but it wasn’t the best either.",

    "Honestly, I was quite disappointed with this event. The organization felt chaotic, and there were constant delays that threw off the schedule. The speakers, while knowledgeable, often went off-topic, making it hard to stay engaged. The venue was poorly chosen, with uncomfortable seating and bad acoustics that made it difficult to hear presentations clearly. The food options were extremely limited, and the networking opportunities were not well structured. I expected a lot more given the hype, but it felt like a waste of time and money.",
    "This event was a disaster from start to finish. The check-in process was slow and disorganized, leading to long wait times. Several sessions were canceled last minute, and there was little communication from the organizers. The speakers were underwhelming, often reading straight from slides rather than engaging with the audience. The venue itself was overcrowded, making it difficult to move around. I left early out of frustration and will definitely not be attending again."

    
]

# ✅ Inference
for review in new_reviews:
    inputs = tokenizer(review, return_tensors="tf", padding=True, truncation=True)
    outputs = model(**inputs)
    pred_label = tf.argmax(outputs.logits, axis=1).numpy()[0]

    sentiment = label_map[pred_label]
    print(f"Review: {review}\nPredicted Sentiment: {sentiment}\n")

Review: Attending this event was an absolute delight! The organizers did a phenomenal job ensuring everything was seamless, from the registration process to the closing ceremony. The speakers were incredibly insightful, offering fresh perspectives that kept the audience engaged. The ambiance was vibrant, and the catering was top-notch. I particularly loved the networking opportunities, as I met professionals who provided valuable insights into my field. Overall, a well-organized, highly enjoyable experience that I would definitely recommend!
Predicted Sentiment: positive

Review: The event had its ups and downs. While I appreciated the effort put into organizing it, I felt some aspects could have been improved. The keynote speakers were great, but the breakout sessions felt repetitive. The venue was decent, though the seating arrangements were a bit cramped. The networking session was useful, but I wish there were more structured ways to interact with other attendees. It wasn’t the wor

In [18]:
model.save("fine_tuned_nlptown_bert_model_ver2", save_format="tf")

INFO:tensorflow:Assets written to: fine_tuned_nlptown_bert_model_ver2\assets


INFO:tensorflow:Assets written to: fine_tuned_nlptown_bert_model_ver2\assets


In [ ]:
##post training quantization
import tensorflow as tf

# ✅ Load the fine-tuned model
model = tf.saved_model.load("fine_tuned_nlptown_bert_model")

# ✅ Convert the model to a TFLite model with quantization
converter = tf.lite.TFLiteConverter.from_saved_model("fine_tuned_nlptown_bert_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# ✅ Save the quantized model
with open("fine_tuned_nlptown_bert_model_quantized.tflite", "wb") as f:
    f.write(tflite_model)

print("🔥 Quantized model saved as TFLite.")


🔥 Quantized model saved as TFLite.


In [9]:
model = tf.saved_model.load("fine_tuned_nlptown_bert_model")

In [10]:
import tensorflow as tf
from transformers import BertTokenizer

# ✅ Load Model
model = tf.saved_model.load("fine_tuned_nlptown_bert_model")
infer = model.signatures["serving_default"]

# ✅ Label Map
label_map = {
    0: 'negative',   # 1-2 stars
    1: 'negative',
    2: 'neutral',     # 3 stars
    3: 'positive',    # 4-5 stars
    4: 'positive'
}

# ✅ Load Tokenizer
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# ✅ New Reviews
new_reviews = [
    "Very Amazing experience i need refund for my ticket",
    "It was an average experience , maybe I'll visit next time",
    "It was fun but food was very bad"
]

# ✅ Inference Loop
for review in new_reviews:
    # Tokenize
    inputs = tokenizer(review, return_tensors="tf", padding=True, truncation=True)

    # Prepare the inputs for the model
    outputs = infer(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        token_type_ids=inputs.get('token_type_ids', tf.zeros_like(inputs['input_ids']))
    )

    # Extract logits and predict
    logits = outputs['logits']
    pred_label = tf.argmax(logits, axis=1).numpy()[0]

    sentiment = label_map[pred_label]
    print(f"Review: {review}\nPredicted Sentiment: {sentiment}\n")


Review: Very Amazing experience i need refund for my ticket
Predicted Sentiment: positive

Review: It was an average experience , maybe I'll visit next time
Predicted Sentiment: neutral

Review: It was fun but food was very bad
Predicted Sentiment: neutral



In [ ]:
## tflite model is not working

interpreter = tf.lite.Interpreter(model_path="fine_tuned_nlptown_bert_model_quantized.tflite")

c:\Users\ASUS\anaconda3\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [11]:
import numpy as np

In [9]:
interpreter.allocate_tensors()

In [16]:
from scipy.special import softmax

In [17]:

# ✅ Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# ✅ Label Map
label_map = {
    0: 'negative',   # 1-2 stars
    1: 'negative',
    2: 'neutral',     # 3 stars
    3: 'positive',    # 4-5 stars
    4: 'positive'
}

# ✅ Load Tokenizer
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# ✅ New Reviews
new_reviews = [
    "It was an average experience, maybe I'll visit next time",
    "It was fun but the food was very bad",
    "Absolutely loved the movie! Great acting and storyline.",
    "Worst customer service ever. Never coming back.",
    "The concert was amazing, had a wonderful time!"
]

# ✅ Check the expected input shape
expected_shape = input_details[0]['shape']  # e.g., (1, 512)
max_len = expected_shape[1]

# ✅ Inference Loop
for review in new_reviews:
    # Tokenize
    inputs = tokenizer(review, return_tensors="tf", padding='max_length', truncation=True, max_length=max_len)

    # Prepare input tensors
    input_ids = inputs['input_ids'].numpy().astype(np.int32)
    attention_mask = inputs['attention_mask'].numpy().astype(np.int32)
    token_type_ids = inputs.get('token_type_ids', tf.zeros_like(inputs['input_ids'])).numpy().astype(np.int32)

    # Resize tensors to match TFLite input shape
    input_ids = np.resize(input_ids, expected_shape)
    attention_mask = np.resize(attention_mask, expected_shape)
    token_type_ids = np.resize(token_type_ids, expected_shape)

    # ✅ Set the input tensors
    interpreter.set_tensor(input_details[0]['index'], input_ids)
    interpreter.set_tensor(input_details[1]['index'], attention_mask)
    interpreter.set_tensor(input_details[2]['index'], token_type_ids)

    # ✅ Run inference
    interpreter.invoke()

    # ✅ Extract logits and apply softmax
    logits = interpreter.get_tensor(output_details[0]['index'])[0]
    probabilities = softmax(logits)

    # ✅ Get the predicted label
    pred_label = np.argmax(probabilities)
    confidence = probabilities[pred_label]

    sentiment = label_map[pred_label]
    
    print(f"🔥 Review: {review}")
    print(f"✅ Predicted Sentiment: {sentiment}")
    print(f"📊 Confidence: {confidence:.4f}")
    print("=" * 100)

We need to remove 14 to truncate the input but the first sequence has a length 13. 
We need to remove 10 to truncate the input but the first sequence has a length 9. 
We need to remove 12 to truncate the input but the first sequence has a length 11. 
We need to remove 10 to truncate the input but the first sequence has a length 9. 
We need to remove 11 to truncate the input but the first sequence has a length 10. 


🔥 Review: It was an average experience, maybe I'll visit next time
✅ Predicted Sentiment: negative
📊 Confidence: 0.2560
🔥 Review: It was fun but the food was very bad
✅ Predicted Sentiment: negative
📊 Confidence: 0.2560
🔥 Review: Absolutely loved the movie! Great acting and storyline.
✅ Predicted Sentiment: negative
📊 Confidence: 0.2560
🔥 Review: Worst customer service ever. Never coming back.
✅ Predicted Sentiment: negative
📊 Confidence: 0.2560
🔥 Review: The concert was amazing, had a wonderful time!
✅ Predicted Sentiment: negative
📊 Confidence: 0.2560
